In [1]:
import pandas as pd
import os

### Notes

Trying to experiment with loading in data programmatically

In [2]:
files = os.listdir('repo_data/')

In [9]:
initial_dfs = []
for dir, subdir, files in os.walk('repo_data/'):
    print("directory", dir)
    # print("subdirectory", subdir)
    # print("files", files)
    for file in files:
        if file.endswith('.csv'):
            # print("file", file)
            df = pd.read_csv(os.path.join(dir, file))
            df['file_path'] = os.path.join(dir, file)
            df['directory'] = dir
            initial_dfs.append(df)

directory repo_data/
directory repo_data/humanities
directory repo_data/public_history
directory repo_data/cultural_analytics
directory repo_data/computational_humanities
directory repo_data/digital_history
directory repo_data/digital_cultural_heritage
directory repo_data/digital_humanities


In [10]:
combined_df = pd.concat(initial_dfs)

/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_54998/3633843701.py:1: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  combined_df = pd.concat(initial_dfs)


In [12]:
combined_df.directory.value_counts()

repo_data/humanities                   3172
repo_data/digital_humanities           2073
repo_data/digital_history               609
repo_data/public_history                521
repo_data/cultural_analytics            104
repo_data/computational_humanities       68
repo_data/digital_cultural_heritage      30
Name: directory, dtype: int64

- Need to subset the data to only include repositories related to teaching